In [1]:
schema_name = "fin_schema"

graph_path = "finbench"
graph_oid = 17368

label_where_key = {
    "account":"nickname",
    "company":"companyname",
    "person":"personname",
    "medium":"mediumtype",
}


exclude_return_keys = set({"type","label"})




In [2]:
list(label_where_key.keys())

['account', 'company', 'person', 'medium']

## 生成medical schmea的语句
```
create graph medical_schema;
set graph_path = medical_schema;

create (n:node {name: 'first_level_departments'});
create (n:node {name: 'second_level_departments'});
create (n:node {name: 'inspection_means'});
create (n:node {name: 'treatment_plan'});
create (n:node {name: 'manufacturer'});
create (n:node {name: 'disease'});
create (n:node {name: 'symptoms'});
create (n:node {name: 'drug'});
create (n:node {name: 'food'});
create (n:node {name: 'recipe'});

match (n:node {name: 'disease'}),(m:node {name: 'drug'}) create (n)-[:relation {name:'highly_rated_medicines'}]->(m);
match (n:node {name: 'disease'}),(m:node {name: 'food'}) create (n)-[:relation {name:'suitable_to_eat'}]->(m);
match (n:node {name: 'second_level_departments'}),(m:node {name: 'first_level_departments'}) create (n)-[:relation {name:'belong_to'}]->(m);
match (n:node {name: 'disease'}),(m:node {name: 'drug'}) create (n)-[:relation {name:'common_used'}]->(m);
match (n:node {name: 'disease'}),(m:node {name: 'disease'}) create (n)-[:relation {name:'complication'}]->(m);
match (n:node {name: 'disease'}),(m:node {name: 'food'}) create (n)-[:relation {name:'avoid_eating'}]->(m);
match (n:node {name: 'disease'}),(m:node {name: 'second_level_departments'}) create (n)-[:relation {name:'affiliation_department'}]->(m);
match (n:node {name: 'disease'}),(m:node {name: 'treatment_plan'}) create (n)-[:relation {name:'treatment'}]->(m);
match (n:node {name: 'manufacturer'}),(m:node {name: 'drug'}) create (n)-[:relation {name:'produce_medicine'}]->(m);
match (n:node {name: 'disease'}),(m:node {name: 'symptoms'}) create (n)-[:relation {name:'with_symptoms'}]->(m);
match (n:node {name: 'disease'}),(m:node {name: 'recipe'}) create (n)-[:relation {name:'recommended_recipes'}]->(m);
match (n:node {name: 'disease'}),(m:node {name: 'inspection_means'}) create (n)-[:relation {name:'diagnostic_tests'}]->(m);

```

In [3]:
## 查询生成 首先处理schema
## 对于agensGRaph 需要手动生成schenma
## 同时 我们认为正过来和倒过去是一个pattren  这儿可以写算法解决的 怎么写呢

## 这里我们只关心属性的查询和 一跳二跳的查询   变长路径查询占用10%



In [4]:
basepatern = "(n1)"

basepatern_set = "n"
pattern_remove_set =set(basepatern_set)
all_pattern = []
all_pattern.append(basepatern)
def generate_pattern(basepatern,basepatern_set,i):
    new_need_iter = []
    new_pattern1 = basepatern+'-[r'+str(i)+']'+'->(n'+str(i+1)+')'
    new_pattern2 = basepatern+'<-[r'+str(i)+']'+'-(n'+str(i+1)+')'

    new_pattern1_set = basepatern_set+'|-r-n'
    new_pattern2_set = basepatern_set+'-r-|n'
    if new_pattern1_set in pattern_remove_set or new_pattern1_set[::-1] in pattern_remove_set:
        pass
    else:
        all_pattern.append(new_pattern1)
        pattern_remove_set.add(new_pattern1_set)
    if new_pattern2_set in pattern_remove_set or new_pattern2_set[::-1] in pattern_remove_set:
        pass
    else:
        all_pattern.append(new_pattern2)        
        pattern_remove_set.add(new_pattern2_set)
    new_need_iter.append(new_pattern1)
    new_need_iter.append(new_pattern2)
    if i < 1:  ## 这里必须确定一个问题， 多跳的自然语言很难描述   所以我们应该把跳数缩短，更复杂的查询用户可以第二次查询
        generate_pattern(new_need_iter[0],new_pattern1_set, i+1)
        generate_pattern(new_need_iter[1],new_pattern2_set, i+1)

## 正过来和倒过去是一个pattern怎么处理 
generate_pattern(basepatern,basepatern_set,1)
print(len(all_pattern))
all_pattern

2


['(n1)', '(n1)-[r1]->(n2)']

```
match (n) return n;
match (n)-[r]->(m) return n,r,m;
match (n1)-[r1]->(m2)-[r2]->(n3) return n1,r1,n2,r2,n3;
```



In [5]:
cypher_schema_query = []
for pattern in all_pattern:
    query = "match " + pattern +" return distinct *"
    cypher_schema_query.append(query)
cypher_schema_query

['match (n1) return distinct *', 'match (n1)-[r1]->(n2) return distinct *']

In [6]:
import re
import json
node_pattern = re.compile(r'(.+?)\[(.+?)\](\{.*?\})', re.S)
edge_pattern = re.compile(r'(.+?)\[(.+?)\]\[(.+?),(.+?)\](\{.*?\})', re.S)
def resolve_node(row_data):
    m = node_pattern.match(row_data)
    label = m.group(1)
    vid = m.group(2)
    props = json.loads(m.group(3))
    return (label, vid, props)

def resolve_edge(row_data):
    m = edge_pattern.match(row_data)
    label = m.group(1)
    eid = m.group(2)
    start = m.group(3)
    end = m.group(4)
    props = json.loads(m.group(5))
    return (label, eid, start, end, props)

def resolve_path(row_data):
    edges = []
    allmatch = edge_pattern.findall(row_data)
    for m in allmatch:
        label = m[0]
        if label.startswith(","):
            label =label[1:]
        eid = m[1]
        start = m[2]
        end = m[3]
        props = json.loads(m[4])
        edges.append( (label, eid, start, end, props))
    print(edges)
    return edges

In [7]:
cypher_schema_query = sorted(cypher_schema_query, key=len)

new_pattern = []
## 对schema进行查询
import psycopg
import random


pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432)
     
       
cur = pg_con.cursor()
cur.execute(f"set graph_path = {schema_name}")
for query in cypher_schema_query:
    cur.execute(query)
    rows = cur.fetchall()
    print("数据的总行数:",len(rows),"\t","查询语句",query)
    for row in rows:
        rate = random.random()
        pattern = ""
        if (len(rows) < 500 and rate < 50/len(rows) )  or (len(rows) > 500 and rate < 10/len(rows) ):
            node_index = 1
            edge_index = 1 
            last_node_id = -1       
            for i in range(len(row)):
                    row_data = str(row[i])
                    is_node = node_pattern.match(row_data)
                    is_edge = edge_pattern.match(row_data)
                    if row_data.startswith('[') and row_data.endswith(']') :
                        row_data = row_data[1:len(row_data)-1]
                        edges = resolve_path(row_data)
                    elif is_edge:
                        edge = resolve_edge(row_data)
                        edge_label = edge[4]['name']
                        edge_start_id = edge[2]
                        if edge_start_id == last_node_id:
                            pattern=pattern+"-[r"+str(edge_index)+":"+edge_label+"]->"
                        else:
                            pattern=pattern+"<-[r"+str(edge_index)+":"+edge_label+"]-"     
                        edge_index = edge_index+1
                    elif is_node:
                        node = resolve_node(row_data)
                        node_label = node[2]['name']
                        last_node_id = node[1]
                        pattern=pattern+"(n"+str(node_index)+":"+node_label+")"
                        node_index = node_index+1
            print(pattern)
            new_pattern.append(pattern)
pg_con.close()
print(len(new_pattern))
# 方法一：使用json.dump()函数保存为JSON格式并写入文件
with open("new_pattern.json", "w") as file:
    json.dump(new_pattern, file,indent=4)

数据的总行数: 5 	 查询语句 match (n1) return distinct *
(n1:account)
(n1:company)
(n1:loan)
(n1:medium)
(n1:person)
数据的总行数: 12 	 查询语句 match (n1)-[r1]->(n2) return distinct *
(n1:account)-[r1:accountrepayloan]->(n2:loan)
(n1:account)-[r1:accounttransferaccount]->(n2:account)
(n1:account)-[r1:accountwithdrawaccount]->(n2:account)
(n1:company)-[r1:companyapplyloan]->(n2:loan)
(n1:company)-[r1:companyguaranteecompany]->(n2:company)
(n1:company)-[r1:companyinvestcompany]->(n2:company)
(n1:loan)-[r1:loandepositaccount]->(n2:account)
(n1:medium)-[r1:mediumsigninaccount]->(n2:account)
(n1:person)-[r1:personapplyLoan]->(n2:loan)
(n1:person)-[r1:personguaranteeperson]->(n2:person)
(n1:person)-[r1:personinvestcompany]->(n2:company)
(n1:person)-[r1:personownaccount]->(n2:account)
17


In [8]:
cypher_querys = []
for pattern in new_pattern:
    cypher_querys.append("match "+pattern+" return * limit 50")
print(len(cypher_querys))    
cypher_querys

17


['match (n1:account) return * limit 50',
 'match (n1:company) return * limit 50',
 'match (n1:loan) return * limit 50',
 'match (n1:medium) return * limit 50',
 'match (n1:person) return * limit 50',
 'match (n1:account)-[r1:accountrepayloan]->(n2:loan) return * limit 50',
 'match (n1:account)-[r1:accounttransferaccount]->(n2:account) return * limit 50',
 'match (n1:account)-[r1:accountwithdrawaccount]->(n2:account) return * limit 50',
 'match (n1:company)-[r1:companyapplyloan]->(n2:loan) return * limit 50',
 'match (n1:company)-[r1:companyguaranteecompany]->(n2:company) return * limit 50',
 'match (n1:company)-[r1:companyinvestcompany]->(n2:company) return * limit 50',
 'match (n1:loan)-[r1:loandepositaccount]->(n2:account) return * limit 50',
 'match (n1:medium)-[r1:mediumsigninaccount]->(n2:account) return * limit 50',
 'match (n1:person)-[r1:personapplyLoan]->(n2:loan) return * limit 50',
 'match (n1:person)-[r1:personguaranteeperson]->(n2:person) return * limit 50',
 'match (n1:pe

In [9]:
## 通过数据库的base_schema pattern 查询数据库 生成带有 where return order by
import re
import json

# 从文件加载JSON数据
with open('new_pattern.json', 'r') as file:
    new_pattern = json.load(file)

cypher_querys = []
for pattern in new_pattern:
    cypher_querys.append("match "+pattern+" return * limit 50")


node_pattern = re.compile(r'(.+?)\[(.+?)\](\{.*?\})', re.S)
edge_pattern = re.compile(r'(.+?)\[(.+?)\]\[(.+?),(.+?)\](\{.*?\})', re.S)

def resolve_node(row_data):
    m = node_pattern.match(row_data)
    label = m.group(1)
    vid = m.group(2)
    props = json.loads(m.group(3))
    return (label, vid, props)

def resolve_edge(row_data):
    m = edge_pattern.match(row_data)
    label = m.group(1)
    eid = m.group(2)
    start = m.group(3)
    end = m.group(4)
    props = json.loads(m.group(5))
    return (label, eid, start, end, props)

def resolve_path(row_data):
    edges = []
    allmatch = edge_pattern.findall(row_data)
    for m in allmatch:
        label = m[0]
        if label.startswith(","):
            label =label[1:]
        eid = m[1]
        start = m[2]
        end = m[3]
        props = json.loads(m[4])
        edges.append( (label, eid, start, end, props))
    print(edges)
    return edges

import psycopg
import random


cypher_base_query = []

# node_func = ["id({})","properties({})","keys({})","count({})"]
# edge_func =  ["id({})","properties({})","keys({})","endNode({})","startNode({})","type({})"]
# string_func= ["rTrim({})","lTrim({})","trim({})","toUpper({})","toLower({})","reverse({})"]

pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432,
            options="-c statement_timeout=10000"
            )
     
pg_con.autocommit=True        
cur = pg_con.cursor()
cur.execute(f"set graph_path = {graph_path}")


for pattern in cypher_querys:
    try:
        cur.execute(pattern)
        rows = cur.fetchall()
        print("数据的总行数:",len(rows))

        base_query_pattern = pattern.replace("return * limit 50","")
        column_names = [desc[0] for desc in cur.description]
        for row in rows:
            where_key = []
            return_key = []
            for i in range(len(row)):
                row_data = str(row[i])
                if "feature_emd" != column_names[i]:
                    var_name = column_names[i]

                    is_node = node_pattern.match(row_data)
                    is_edge = edge_pattern.match(row_data)
                    if row_data.startswith('[') and row_data.endswith(']') :
                        row_data = row_data[1:len(row_data)-1]
                        edges = resolve_path(row_data)
                    elif is_edge:
                        edge = resolve_edge(row_data)
                        properties = edge[4]
                        keys = list(properties.keys())
                        for key in keys:
                            return_key.append(var_name+"."+key)

                    elif is_node:
                        node = resolve_node(row_data)
                        node_label = node[0]
                        properties = node[2]
                        keys = list(properties.keys())
                        for key in keys:
                            if key not in exclude_return_keys:
                                return_key.append(var_name+"."+key)
                        ### where 条件的拼接只出现在这里
                        if node_label in label_where_key:
                            where_key.append(var_name+"."+label_where_key[node_label]+"='"+properties[label_where_key[node_label]]+"'" )
                        
           

            ## 返回值
            return_key = random.sample(return_key,random.randint(1,len(return_key)) )
            if len(where_key) >0:
                where_key = random.sample(where_key,random.randint(1,len(where_key)) )

            if len(return_key) > 1:
                return_param = ", ".join(return_key)
            else:
                return_param =   return_key[0]         
            
            return_param = " return "+ return_param 


                        # order by 
            order_param = ""
            if random.random()<0.3:
                which_could_order_by =[key for key in return_key 
                                       if key.endswith('.name') 
                                      ]
                if len(which_could_order_by) >0:
                    order_param = order_param + " order by "+ which_could_order_by[random.randint(0,len(which_could_order_by)-1)]
                    if random.random()<0.3:
                        order_param = order_param + " desc"

            
            
            where_param = ''

            if len(where_key) > 1:
                where_param = " and ".join(where_key)
            elif len(where_key) == 1:
                where_param = where_key[0]
            else:    
                continue
            where_param = " where "+where_param    

            cypher_base_query.append(base_query_pattern+where_param+return_param + order_param)         
    except Exception as e:
        print("-------------------------",e)
pg_con.close()

print(len(cypher_base_query))
# 方法一：使用json.dump()函数保存为JSON格式并写入文件
with open("cypher_base_query.json", "w") as file:
    json.dump(cypher_base_query, file,indent=4)

数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 50
数据的总行数: 0
数据的总行数: 0
数据的总行数: 50
数据的总行数: 50
700


In [10]:
### 接着处理语句  添加limit 和 skip 关键字  t同时替换掉一些非法的属性值 agens 区分大小写 
### 赋予30的概率吧

import json
import random
# 从文件加载JSON数据
cypher_base_with_limit_skip_query=[]
with open('cypher_base_query.json', 'r') as file:
    cypher_base_query = json.load(file)
for base_query in  cypher_base_query:
    base_query = base_query.replace("   "," ")

    if random.random()<0.3:
       
        if random.random()<0.3:
            skip = random.randint(1,20)    
            base_query = base_query + " skip " + str(skip)  + " limit " + str(random.randint(1,20))
        else:
            base_query = base_query + " limit " + str(random.randint(1,20)) 
        cypher_base_with_limit_skip_query.append(base_query)    
    else:
        cypher_base_with_limit_skip_query.append(base_query)

    
print(len(cypher_base_with_limit_skip_query))
# 方法一：使用json.dump()函数保存为JSON格式并写入文件
with open("cypher_base_with_limit_skip_query.json", "w") as file:
    json.dump(cypher_base_with_limit_skip_query, file,indent=4)

700


In [11]:
#### 过滤掉没有结果的值和查询超时的值
## 对schema进行查询
import psycopg
cypher_base_with_limit_skip_query_filter_none = []
pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432,
            options="-c statement_timeout=10000")
     
pg_con.autocommit=True       
cur = pg_con.cursor()
cur.execute(f"set graph_path = {graph_path}")
for query in cypher_base_with_limit_skip_query:
    try:
        cur.execute(query)
        rows = cur.fetchall()
        print("数据的总行数:",len(rows),"\t","查询语句",query)
        if len(rows) > 0:
            cypher_base_with_limit_skip_query_filter_none.append(query)
    except Exception as e:
        print(e)       

pg_con.close()

print(len(cypher_base_with_limit_skip_query_filter_none))
# 方法一：使用json.dump()函数保存为JSON格式并写入文件
with open("cypher_base_with_limit_skip_query_filter_none.json", "w") as file:
    json.dump(cypher_base_with_limit_skip_query_filter_none, file,indent=4)

数据的总行数: 1 	 查询语句 match (n1:account)  where n1.nickname='Renato Holness' return n1.createtime, n1.accountlevel, n1.lastlogintime, n1.nickname, n1.phonenum
数据的总行数: 1 	 查询语句 match (n1:account)  where n1.nickname='Kurt Fernandes' return n1.createtime, n1.freqlogintype, n1.nickname, n1.accountid, n1.phonenum, n1.email
数据的总行数: 2 	 查询语句 match (n1:account)  where n1.nickname='Laverna Model' return n1.lastlogintime, n1.accouttype, n1.createtime, n1.email, n1.phonenum, n1.accountid
数据的总行数: 2 	 查询语句 match (n1:account)  where n1.nickname='Peggy Gayheart' return n1.phonenum, n1.accouttype, n1.email
数据的总行数: 0 	 查询语句 match (n1:account)  where n1.nickname='Eleni Mccleve' return n1.lastlogintime, n1.email, n1.freqlogintype, n1.createtime, n1.isblocked skip 5 limit 14
数据的总行数: 0 	 查询语句 match (n1:account)  where n1.nickname='Bethann Fasching' return n1.phonenum, n1.accountlevel, n1.createtime skip 5 limit 2
数据的总行数: 2 	 查询语句 match (n1:account)  where n1.nickname='Mayra Minkoff' return n1.isblocked, n1.freq

In [12]:
import re
import json
node_pattern = re.compile(r'(.+?)\[(.+?)\](\{.*?\})', re.S)
edge_pattern = re.compile(r'(.+?)\[(.+?)\]\[(.+?),(.+?)\](\{.*?\})', re.S)
def resolve_node(row_data):
    m = node_pattern.match(row_data)
    label = m.group(1)
    vid = m.group(2)
    props = json.loads(m.group(3))
    return (label, vid, props)

def resolve_edge(row_data):
    m = edge_pattern.match(row_data)
    label = m.group(1)
    eid = m.group(2)
    start = m.group(3)
    end = m.group(4)
    props = json.loads(m.group(5))
    return (label, eid, start, end, props)

def resolve_path(row_data):
    edges = []
    allmatch = edge_pattern.findall(row_data)
    for m in allmatch:
        label = m[0]
        if label.startswith(","):
            label =label[1:]
        eid = m[1]
        start = m[2]
        end = m[3]
        props = json.loads(m[4])
        edges.append( (label, eid, start, end, props))
    print(edges)
    return edges

schema_ldbc_info = {}

edge_schema = []
node_schema = []
## 对schema进行查询
import psycopg

pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432)
     
nodes = set()

edge_query = 'match (n)-[r]->(m) return n,r,m'

cur = pg_con.cursor()
cur.execute(f"set graph_path = {schema_name}")
### 节点类型
cur.execute(edge_query)
rows = cur.fetchall()

for row in rows:

    edge_desc= ""
    start_label_desc = ""
    end_label_desc = ""
    for i in range(len(row)):
            row_data = str(row[i])
            is_node = node_pattern.match(row_data)
            is_edge = edge_pattern.match(row_data)
            if is_edge:
                edge = resolve_edge(row_data)
                edge_label = edge[4]['name']
                edge_desc = '边的类型为：'+ edge_label

            elif is_node:
                node = resolve_node(row_data)
                node_label = node[2]['name']
                nodes.add(node_label)
                if i > 1:
                    end_label_desc = "终点类型为:"+ node_label
                else:
                    start_label_desc = "起点类型为:"+ node_label  
    edge_schema.append(edge_desc+','+start_label_desc+','+end_label_desc)
  

schema_ldbc_info['edges'] = edge_schema

cur.execute(f"set graph_path = {graph_path}")
for node in list(nodes):
    query = 'match (n:'+node+') return keys(n) limit 1'   
    ### 节点类型
    cur.execute(query)
    rows = cur.fetchall()
    for row in rows:
        node_schema.append("节点类型:"+node+',包含属性信息:' + str(row))

schema_ldbc_info['nodes'] = node_schema

# 方法一：使用json.dump()函数保存为JSON格式并写入文件
with open(f"schema_{graph_path}_info.json", "w") as file:
    json.dump(schema_ldbc_info, file,indent=4,ensure_ascii=False)
print(schema_ldbc_info)
pg_con.close()     
 

{'edges': ['边的类型为：accounttransferaccount,起点类型为:account,终点类型为:account', '边的类型为：accountwithdrawaccount,起点类型为:account,终点类型为:account', '边的类型为：loandepositaccount,起点类型为:loan,终点类型为:account', '边的类型为：mediumsigninaccount,起点类型为:medium,终点类型为:account', '边的类型为：personownaccount,起点类型为:person,终点类型为:account', '边的类型为：companyguaranteecompany,起点类型为:company,终点类型为:company', '边的类型为：companyinvestcompany,起点类型为:company,终点类型为:company', '边的类型为：personinvestcompany,起点类型为:person,终点类型为:company', '边的类型为：accountrepayloan,起点类型为:account,终点类型为:loan', '边的类型为：companyapplyloan,起点类型为:company,终点类型为:loan', '边的类型为：personapplyLoan,起点类型为:person,终点类型为:loan', '边的类型为：personguaranteeperson,起点类型为:person,终点类型为:person'], 'nodes': ["节点类型:medium,包含属性信息:(['mediumId', 'isblocked', 'risklevel', 'createtime', 'mediumtype', 'lastlogintime'],)", "节点类型:person,包含属性信息:(['city', 'gender', 'country', 'birthday', 'personId', 'isBlocked', 'createtime', 'personname'],)", "节点类型:account,包含属性信息:(['email', 'nickname', 'phonenum', 'accountid', 'isblocked', 'a

### 这时候应该把上边的数据通过大的语言模型弄出自然语句查询来
### 大模型生成问答对之后再加入向量的上下文信息
### 执行下边这个代码的同时 可以执行第三个文件 生成通义千问的问答向量

In [20]:
# 安装 pip install -U dashscope

import json
import os
import dashscope

 

# 方法一：使用json.dump()函数保存为JSON格式并写入文件
with open("cypher_base_with_limit_skip_query_filter_none.json", "r") as file:
    cypher_base_with_limit_skip_query = json.load(file) 
    
cypher_querys = random.sample(cypher_base_with_limit_skip_query,240)

query_pair = []


for query in cypher_querys:
    response = dashscope.Generation.call(
    # 若没有配置环境变量，请用百炼API Key将下行替换为：api_key="sk-xxx",
    api_key='sk-8f8adba56edf4a29a185fb57c5808a1e',
    model="qwen-plus-2024-09-19", # 模型列表：https://help.aliyun.com/zh/model-studio/getting-started/models
    messages= [
        {"role": "user", "content": "我正在开发一个自然语言转Cypher的接口，但是我没有自然语言描述，你现在是一名Cypher图查询语言专家，你需要将我的Cypher查询语言转换为自然语言描述,使用中文描述,明确返回值所属的变量，只返回描述即可，不需要过多解释，一定要符合人类的自然语言描述习惯、有些查询不需要指明节点的label信息"},
        {"role": "assistant", "content": "好的"},
        {"role": "user", "content": "match (n1:forum)  where n1.title='Album 1 of Hossein Forouhar' return n1.id, n1.\"creationDate\", n1.title"},
        {"role": "assistant", "content": "查找Album 1 of Hossein Forouhar的id,创建日期和他的标题"},
        {"role": "user", "content": "match (n1:forum)-[r1:hastagforum]->(n2:tag)  where n1.title='Wall of Wojciech Ciesla' return n2.id, n2.name"},
        {"role": "assistant", "content": "查找Wall of Wojciech Ciesla的具有的标签信息的id和名称"},
            {"role": "user", "content": query},
    ],
    result_format='message'
    )
    print(response)
    nl_query = response.output.choices[0].message.content
    print(nl_query)
    print(query)
    query_pair.append({nl_query:query})

# 方法一：使用json.dump()函数保存为JSON格式并写入文件
with open("tongyi_query_pair_query.json", "w", encoding="utf-8") as file:
    json.dump(query_pair, file,indent=4,ensure_ascii=False)

{"status_code": 200, "request_id": "db82a1f6-b75c-929c-902b-980fbe554609", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "查找Flachi投资的公司Little-Bergstrom的业务范围、公司名称、Flachi的创建时间、公司的国家和网址以及Flachi的性别"}}]}, "usage": {"input_tokens": 302, "output_tokens": 32, "total_tokens": 334}}
查找Flachi投资的公司Little-Bergstrom的业务范围、公司名称、Flachi的创建时间、公司的国家和网址以及Flachi的性别
match (n1:person)-[r1:personinvestcompany]->(n2:company)  where n2.companyname='Little-Bergstrom' and n1.personname='Flachi' return n2.business, n2.companyname, n1.createtime, n2.country, n2.url, n1.gender
{"status_code": 200, "request_id": "c419f852-b4df-9426-b07f-17be893bcafd", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "stop", "message": {"role": "assistant", "content": "查找名为Night的人对Herman-Bechtelar公司的投资比例，以及Night所在的城市、国家、是否被封禁，Herman-Bechtelar公司的网址、描述、国家、创建时间、公司名称和业务领域。"}}

In [ ]:
### 接着处理数据集  生成自然语言描述的embedding
node_label_names = list(label_where_key.keys())


import psycopg
import json
from pgvector.psycopg import register_vector
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import DirectoryLoader
import numpy as np

model_name = "/home/work/liuytest/demo/maidalun1020/bce-embedding-base_v1"
model_kwargs = {'device': 'npu:0'}
encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity
model = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

def find_real_prompt_from_pg_vector(user_query):
    pg_con = psycopg.connect(
            dbname="postgres",
            host="localhost",
            user="omm",
            password="ZAQ1234@esz",
            port=5432)
     
     
    pg_con.autocommit=True       
    cur = pg_con.cursor()
    register_vector(pg_con)

    embeddings = model.embed_documents(
        [
            user_query
        ]
    )
    embedding = embeddings[0]
    pg_con.execute(f'SET graph_path  = {graph_path};' )
    pg_con.execute('SET hnsw.ef_search = 100;' )
    embedding = np.asarray(embedding)
    datas = []

    prompt = "已知数据库中存在以下数据信息:\n"
    found = False
    for node in node_label_names:
        # pg_con.execute(f'drop  TABLE IF  EXISTS  zyykg_data_{node} cascade') match (n:person) return  n.feature_emd as emb  order by   (emb <#> %s)* -1 desc  LIMIT 1
        data = pg_con.execute(f'match (n:{node}) where (n.feature_emd::vector <#> %s) < -0.5  return  n ,(n.feature_emd::vector <#> %s) as score order by  score asc  LIMIT 3', (embedding,embedding,)).fetchall()
        if len(data)>0:
            found = True
            for data_inrow in data:
                dn_node = resolve_node(data_inrow[0])
                prompt= prompt+f"label为{dn_node[0]},属性{label_where_key[node]}='{dn_node[2][label_where_key[node]]}'的节点\n"  
    if found:
        return prompt 
    else:
        return "" 



key_values = None
with open("tongyi_query_pair_query.json", 'r',encoding="utf-8") as json_file:
    key_values = json.load(json_file) 

new_query_pair_train = []
for row in key_values:
    new_row ={}
    for key,value in row.items():
        user_query = key
        data_prompt = find_real_prompt_from_pg_vector(user_query)
        print(user_query)
        print(data_prompt)
        new_row["instruction"] = data_prompt+"\n" + "请将以下自然语言翻译为对该数据库的Cypher查询:\n"+user_query
        new_row["output"]=value
        new_query_pair_train.append(new_row)
    if len(data_prompt) > 0:
        print("----------------------------------------------------------------------------------------------------")
        print(user_query)
        print(data_prompt)
        print("----------------------------------------------------------------------------------------------------")

with open('tongyi_query_pair_query_train.json', 'w',encoding="utf-8") as json_file:
    json.dump(new_query_pair_train, json_file ,ensure_ascii=False,indent=4)   

查找昵称为Renato Holness的账号的邮箱、常用登录方式、账号ID、账号等级、账号类型、昵称、创建时间、是否被封禁、手机号和最后登录时间
已知数据库中存在以下数据信息:
label为account,属性nickname='Renato Holness'的节点

----------------------------------------------------------------------------------------------------
查找昵称为Renato Holness的账号的邮箱、常用登录方式、账号ID、账号等级、账号类型、昵称、创建时间、是否被封禁、手机号和最后登录时间
已知数据库中存在以下数据信息:
label为account,属性nickname='Renato Holness'的节点

----------------------------------------------------------------------------------------------------
查找昵称为Kurt Fernandes的账号的电话号码、账号类型、是否被封禁、昵称、创建时间、电子邮件、最后登录时间和账号ID，最多返回14条记录。
已知数据库中存在以下数据信息:
label为account,属性nickname='Kurt Fernandes'的节点

----------------------------------------------------------------------------------------------------
查找昵称为Kurt Fernandes的账号的电话号码、账号类型、是否被封禁、昵称、创建时间、电子邮件、最后登录时间和账号ID，最多返回14条记录。
已知数据库中存在以下数据信息:
label为account,属性nickname='Kurt Fernandes'的节点

----------------------------------------------------------------------------------------------------
查找Laverna Model的账户类型、昵称、电子邮件、账户ID、最后登录时间、常用登录类型、创建时间